# Zero Knowledge Proofs

#### What is a Zero Knowledge Proof?

This notebook introduces Zero Knowledge Proofs (ZKPs).

Zero Knowledge Proofs are a way to demonstrate that something is true, without revealing any secret information about the method.

These approaches become very useful in the field of privacy and security for things like authentication and verification of data. For example, you may want to prove that you're over 18, without revealing your exact age.

#### How does it work?

To explain how this works, we'll start with a simple illustrative example, to explain the core principles behind ZKPs.
We'll then explore a more complex example, to show a more full and complete example of how ZKPs work.
Finally, we'll show a real-world application of ZKPs for age verification.

By the end, you'll not only understand how these remarkable protocols work, but also how they can be implemented. They're a remarkable accomplishment, showing that it's possible to maintain confidentiality while establishing trust - a seemingly paradoxical achievement that makes ZKPs one of cryptography's most elegant innovations.

Let's begin with our first example: proving you can add two-digit numbers correctly without revealing how you did it (you can imagine this is a secret algorithm you've been taught, and you're trying to prove you know it without revealing the algorithm).

In [159]:
from abc import ABC, abstractmethod
import random
import hashlib
from ortools.sat.python import cp_model

# Bokeh imports
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool

In [160]:
output_notebook()

Loading BokehJS ...

In [161]:
random.seed(0)  # For reproducibility

## Challenger, Solver & Verifier

Suppose a teacher wants to check if a student can add two-digit numbers correctly.

* The teacher will set the student a set of two-digit addition problems (Challenger). 
* The student will then solve the problems (Solver), 
* The teacher will then check the student's answers (Verifier).

If the student is able to solve all of the teacher's problems, then the teacher is confident that the student can add two-digit numbers correctly. The more problems that are set, the more confident the teacher can become.

If however, the student is not able to solve all of the problems, then the teacher can catch the student, without the method ever being shared.

In [162]:
class BaseChallenger(ABC):
    @abstractmethod
    def challenge(self):
        """
        Produce a new 'challenge' – in this case, two numbers to be added.
        """
        pass


class BaseSolver(ABC):
    @abstractmethod
    def solve(self, challenge):
        """
        Given a challenge (e.g., two numbers), produce a solution (sum).
        Possibly faulty or correct, depending on implementation.
        """
        pass


class BaseVerifier(ABC):
    @abstractmethod
    def verify(self, challenge, solution):
        """
        Given the challenge and a solution, verify correctness.
        Return True if correct, False otherwise.
        """
        pass

## Two Digit Addition

Below we're going to simulate a problem where the student can only solve two digit addition when the first number is less than 90. If the first number is 90 or greater, the student will solve the problem incorrectly. 

This demonstrates that with enough problems set, the teacher can detect that the student is not able to solve all two digit problems accurately, only a subset.

In [163]:
# This class holds logic to sets challenges
class TwoDigitChallenger(BaseChallenger):
    """
    The challenge in this case is to generates two-digit integer addition problems:
    e.g., challenge() -> (12, 45)
    """
    def challenge(self):
        # Generate two random two-digit numbers [0..99]
        a = random.randint(0, 99)
        b = random.randint(0, 99)
        return (a, b)

# This class holds logic to solve the challenge. 
# This is logic is what's "secret" and not known to the challenger.
class TwoDigitSolver(BaseSolver):
    """
    In this case, our solver is flawed. 
    It correctly solves the challenge, not if the first number >= 90,
    in which case it intentionally returns a wrong answer. 
    We do this to illustrate how the solver can be caught if the verifier runs enough trials.
    """
    def solve(self, challenge, n=90):
        a, b = challenge
        if a < n:
            return a + b 
        # Deliberate error if a >= 90 
        else:
            return a + b + 1 # off by one


class TwoDigitVerifier(BaseVerifier):
    """
    Verifies the correctness of an addition result.
    """
    def verify(self, challenge, solution):
        a, b = challenge
        return (solution == a + b)

In [164]:
def run_T_consecutive_trials(challenger, solver, verifier, T):
    """
    Perform T consecutive challenges in one "experiment."
    Return True if solver was correct on all T, else False.
    """
    for _ in range(T):
        problem = challenger.challenge()
        answer = solver.solve(problem, n=90)
        if not verifier.verify(problem, answer):
            return False  
    return True 

In [165]:
def simulate_consecutive_perfect_probability(challenger, solver, verifier, T, experiments=1000):
    """
    For a fixed T, run 'experiments' independent blocks of T challenges.
    Each block must be 100% correct to count as success.
    Return fraction that were perfect.
    """
    successes = 0
    for _ in range(experiments):
        if run_T_consecutive_trials(challenger, solver, verifier, T):
            successes += 1
    return successes / experiments


def plot_probability_of_perfection_vs_T(challenger, solver, verifier,
                                        max_T=10,
                                        experiments_per_T=1000,
                                        output_html="consecutive_perfect.html",
                                        title="Probability of Perfect Answers for T Consecutive Trials"):
    """
    For T in [1..max_T], estimate probability that solver is perfect
    on all T consecutive trials (in a block). Plot those probabilities.
    Includes hover-over tooltips for interactive inspection.
    """
    x_vals = []
    y_vals = []

    for T in range(1, max_T + 1):
        prob = simulate_consecutive_perfect_probability(challenger, solver, verifier, T, experiments=experiments_per_T)
        x_vals.append(T)
        y_vals.append(prob)
        print(f"T={T}, Probability of perfect = {prob:.3f}")

    p = figure(
        title=title,
        x_axis_label="T (Consecutive Trials)",
        y_axis_label="Probability of All-Correct in T Trials",
        width=700,
        height=400
    )

    # Convert data to a ColumnDataSource so we can use hover
    source = ColumnDataSource(data=dict(x=x_vals, y=y_vals))

    # Plot line and circle
    p.line('x', 'y', source=source, line_width=2, color="green", legend_label="All-Correct Probability")
    p.circle('x', 'y', source=source, size=6, fill_color="white", color="green")

    # Add interactive hover
    hover = HoverTool(tooltips=[
        ("T (Trials)", "@x"),
        ("Probability", "@y{0.000}")
    ], mode='vline')
    p.add_tools(hover)

    p.legend.location = "top_right"
    show(p)

### Demonstration of ZKP with Two Digit Addition

In this case, the teacher has set the challenge of solving two-digit addition problems.
However, our solver doesn't solve the problem correctly for all cases, for example if the first number is 90 or greater.

It is then down to the teacher to check if the solver is able to solve the problem correctly.

With enough trials, the teacher can catch the solver out, and prove that the solver is not able to solve the problem correctly, or if the solver does solve all of the problems correctly, the teacher can be confident that the solver knows the secret method.

It is very important that the challenges are set randomly. This ensures that the solver cannot predict the challenge, and therefore cannot cheat.

Randomness also provides diversity of the challenges, so that if the solver does not have a complete proof, the challenges should be able to weed out these cases provided enough successive trials.  

#### Two Digit Solver Trials

In the below simulation, we show the empirically, the probability that the student is able to pass for a given number of trials using our Zero Proof Knowledge approach.

In [166]:
# Instantiate challenger, solver, verifier
challenger = TwoDigitChallenger()
solver = TwoDigitSolver()
verifier = TwoDigitVerifier()

plot_probability_of_perfection_vs_T(
    challenger, solver, verifier,
    max_T=50, #Adjust the number of trials to run
    experiments_per_T=10000, #How many times to run each trial (the higher number, the more precise the probability using Central Limit Theorem)
    output_html="consecutive_perfect.html",
    title="Probability Solver is Perfect in T Consecutive Trials (Emprical)"
)

T=1, Probability of perfect = 0.898
T=2, Probability of perfect = 0.811
T=3, Probability of perfect = 0.733
T=4, Probability of perfect = 0.656
T=5, Probability of perfect = 0.585
T=6, Probability of perfect = 0.526
T=7, Probability of perfect = 0.477
T=8, Probability of perfect = 0.430
T=9, Probability of perfect = 0.388
T=10, Probability of perfect = 0.353
T=11, Probability of perfect = 0.323
T=12, Probability of perfect = 0.280
T=13, Probability of perfect = 0.253
T=14, Probability of perfect = 0.227
T=15, Probability of perfect = 0.210
T=16, Probability of perfect = 0.184
T=17, Probability of perfect = 0.171
T=18, Probability of perfect = 0.148
T=19, Probability of perfect = 0.132
T=20, Probability of perfect = 0.125
T=21, Probability of perfect = 0.113
T=22, Probability of perfect = 0.097
T=23, Probability of perfect = 0.087
T=24, Probability of perfect = 0.083
T=25, Probability of perfect = 0.073
T=26, Probability of perfect = 0.068
T=27, Probability of perfect = 0.060
T=28, Prob

## Graph Coloring Problem

Now we're going to look at a more complex problem, the graph coloring problem.

In this problem, the teacher has set the challenge of coloring a graph with a given number of colors (k), ensuring that no two nodes in the graph that share the same edge, have the same color. In this case, we'll allow three colors and the below graph.

The student claims they can solve a graph coloring problem, and the teacher then wants to check if the student is able to solve the problem correctly, but without revealing the full solution or method.


In [167]:
#Graph
edges = [(0,1), (0,2), (1,3), (2,3), (3,4), (4,5), (3,5), (3,6), (4,6)]
num_nodes = 7
k = 3

In [168]:
# Create a NetworkX graph
G = nx.Graph()
G.add_edges_from(edges)

# Initialize the ipycytoscape widget
cyto_widget = ipycytoscape.CytoscapeWidget()

# Convert the NetworkX graph to ipycytoscape’s format
cyto_widget.graph.add_graph_from_networkx(G)

# Optionally tweak the style
cyto_widget.set_style([
    {
        'selector': 'node',
        'style': {
            'content': 'data(id)',
            'background-color': '#0074D9',
            'color': '#fff',
            'text-valign': 'center'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': '#888'
        }
    }
])

# Now display it in the notebook
cyto_widget

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …

The student's task is to color the graph with the given number of colors, k=3, without having two nodes with the same edge use the same color.

The student will solve the problem, but will not reveal their method to the teacher. Instead they'll just share enough information to show that they know the method, i.e. that any two nodes connected by an edge, have different colors. This doesn't provide any new information about how to solve the problem to the teacher.

In [169]:
import colorsys

#helper for the graph coloring visualisation
def generate_color_palette(k):
    """
    Generates k distinct colors around the HSV color circle.
    Returns a list of hex color strings, e.g. ['#cc33cc', '#33cccc', ...]
    """
    palette = []
    for i in range(k):
        hue = i / max(k, 1)
        saturation = 0.6
        value = 0.9
        r, g, b = colorsys.hsv_to_rgb(hue, saturation, value)
        palette.append(f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}")
    return palette

class GraphColorChallenger(BaseChallenger):
    """
    Challenger that accepts a predefined graph (edges, num_nodes) and a color limit (k).
    """
    def __init__(self, edges, num_nodes, num_colors):
        self.edges = edges
        self.num_nodes = num_nodes
        self.num_colors = num_colors

    def challenge(self):
        """
        Returns a dictionary describing the graph coloring challenge.
        """
        return {
            "num_nodes": self.num_nodes,
            "edges": self.edges,
            "num_colors": self.num_colors
        }


class GraphColorSolver(BaseSolver):
    """
    - solve(challenge): use OR-Tools CP-SAT to find a valid coloring with up to k colors.
    - shuffle_coloring(): permute color labels for zero-knowledge.
    - commit_coloring(): produce a cryptographic commitment for each node's color.
    - open_edge_colors(edge): reveal the color + salt for two endpoints of that edge.
    """

    def __init__(self):
        self.graph_info = None
        self.original_coloring = None  # final integer color assignment for each node
        self.permuted_labels = None
        self.k = 0

        # For ZKP commitments
        self.commitments = []
        self.salts = []

    def solve(self, challenge):
        """
        Solve the graph k-coloring feasibility problem using CP-SAT:
         - Each node has an integer var in [0..k-1]
         - For each edge (u,v), color[u] != color[v]
         - If feasible, store the coloring. Otherwise, raise an error.
        """
        self.graph_info = challenge
        self.k = challenge["num_colors"]
        n = challenge["num_nodes"]
        edges = challenge["edges"]

        coloring = self._solve_with_cp_sat(n, edges, self.k)
        if coloring is None:
            raise ValueError(f"No valid {self.k}-coloring found for the given graph.")
        self.original_coloring = coloring

        # Initialize a random permutation of color labels
        self._generate_new_label_permutation()

    def _solve_with_cp_sat(self, n, edges, k):
        """
        Use OR-Tools CP-SAT to check if the graph can be colored with up to k colors.
        - We only do a feasibility check: color[i] in [0..k-1]
        - For each edge (u,v), color[u] != color[v]
        - If feasible, return a valid coloring list. Otherwise, return None.
        """
        model = cp_model.CpModel()

        # Create integer variables for each node in [0..k-1]
        color_vars = [model.NewIntVar(0, k - 1, f'color_{i}') for i in range(n)]

        # Adjacency constraints
        for (u, v) in edges:
            model.Add(color_vars[u] != color_vars[v])

        # Solve
        solver = cp_model.CpSolver()
        status = solver.Solve(model)
        if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
            return [solver.Value(color_vars[i]) for i in range(n)]
        else:
            return None

    def shuffle_coloring(self):
        """Randomly permute color labels for zero-knowledge each round."""
        self._generate_new_label_permutation()

    def commit_coloring(self):
        """
        Produce a commitment (SHA-256 hash of random salt + color) for each node
        under the current permuted labeling.
        """
        n = self.graph_info["num_nodes"]
        self.commitments = []
        self.salts = []

        for node in range(n):
            real_color = self.original_coloring[node]
            perm_color = self.permuted_labels[real_color]
            # Random salt
            salt = random.getrandbits(128).to_bytes(16, 'big')
            # Create hash input
            hash_input = salt + perm_color.to_bytes(4, 'big')
            commit_hash = hashlib.sha256(hash_input).hexdigest()

            self.commitments.append(commit_hash)
            self.salts.append(salt)

        return self.commitments

    def open_edge_colors(self, edge):
        """
        Reveal (nodeIndex, permuted_color, salt) for both endpoints of the edge.
        """
        (u, v) = edge
        color_u = self._get_permuted_color(u)
        color_v = self._get_permuted_color(v)
        salt_u = self.salts[u]
        salt_v = self.salts[v]
        return (u, color_u, salt_u), (v, color_v, salt_v)

    def _generate_new_label_permutation(self):
        label_list = list(range(self.k))
        random.shuffle(label_list)
        self.permuted_labels = label_list

    def _get_permuted_color(self, node):
        real_c = self.original_coloring[node]
        return self.permuted_labels[real_c]
    
    def visualize_coloring(self, use_permuted=False):
        """
        Returns an ipycytoscape widget showing each node in its color.
        
        By default, it uses the *original* color assignment.
        If 'use_permuted=True', it applies the currently stored 'permuted_labels'
        to each node's original color (which might have been generated in a shuffle).
        
        Works around older ipycytoscape versions by assigning node colors 
        *after* loading the graph.
        """
        import networkx as nx
        import ipycytoscape
        import colorsys

        def generate_color_palette(k):
            """
            Generates k distinct colors around the HSV color circle.
            Returns a list of hex color strings, e.g. ['#cc33cc', '#33cccc', ...].
            """
            palette = []
            for i in range(k):
                hue = i / max(k, 1)
                saturation = 0.6
                value = 0.9
                r, g, b = colorsys.hsv_to_rgb(hue, saturation, value)
                palette.append(f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}")
            return palette

        n = self.graph_info["num_nodes"]
        edges = self.graph_info["edges"]

        # Create a NetworkX graph
        G = nx.Graph()
        G.add_nodes_from(range(n))
        G.add_edges_from(edges)

        # Generate a distinct color palette for k colors
        color_palette = generate_color_palette(self.k)

        # Assign each node's color to a node attribute
        for node in G.nodes:
            original_color = self.original_coloring[node]
            
            if use_permuted:
                # If we haven't shuffled or 'permuted_labels' doesn't exist, handle gracefully
                if not hasattr(self, 'permuted_labels') or self.permuted_labels is None:
                    raise ValueError("Cannot visualize permuted labels; 'permuted_labels' not set. Call shuffle_coloring() first.")
                permuted_label = self.permuted_labels[original_color]
                assigned_color = color_palette[permuted_label]
            else:
                assigned_color = color_palette[original_color]
            
            G.nodes[node]['color'] = assigned_color

        # Create the Cytoscape widget
        cyto_widget = ipycytoscape.CytoscapeWidget()
        cyto_widget.graph.add_graph_from_networkx(G)

        # Manually copy each node's 'color' attribute from G into the ipycytoscape node data
        for node in cyto_widget.graph.nodes:
            node_id_str = node.data['id']       # e.g. "0"
            node_id_int = int(node_id_str)      # your original node index is an integer
            node.data['color'] = G.nodes[node_id_int]['color']

        # Define a style to color each node background using 'data(color)'
        style = [
            {
                'selector': 'node',
                'style': {
                    'content': 'data(id)',
                    'background-color': 'data(color)',
                    'color': '#fff',  # node label color
                    'text-valign': 'center'
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'line-color': '#999'
                }
            }
        ]
        cyto_widget.set_style(style)

        return cyto_widget

class GraphColorVerifier(BaseVerifier):
    """
    - verify(...) remains a placeholder for a single-step check.
    - We'll do multiple rounds: pick an edge, check commitments, etc.
    """

    def __init__(self):
        self.last_commitments = []  # store solver's commitments each round
        self.graph_info = None

    def verify(self, challenge, solution):
        # Not used in partial ZKP approach. Always True as a placeholder.
        return True

    def pick_edge_for_check(self):
        edges = self.graph_info["edges"]
        return random.choice(edges)

    def store_commitments(self, commitments):
        self.last_commitments = commitments

    def verify_opened_edge(self, opened_u, opened_v):
        """
        Check that:
         1) The committed hash matches the revealed color+salt for each node.
         2) The two revealed colors differ.
        """
        node_u, color_u, salt_u = opened_u
        node_v, color_v, salt_v = opened_v

        commit_u = self._compute_hash(salt_u, color_u)
        commit_v = self._compute_hash(salt_v, color_v)

        # Check if re-hashed values match stored commitments
        if commit_u != self.last_commitments[node_u]:
            return False
        if commit_v != self.last_commitments[node_v]:
            return False

        # The two colors must differ
        return (color_u != color_v)

    def _compute_hash(self, salt, color):
        hash_input = salt + color.to_bytes(4, 'big')
        return hashlib.sha256(hash_input).hexdigest()


# 5. Multi-Round ZKP Demo with Commitments
def zero_knowledge_proof_demo_with_commitments(challenger, solver, verifier, rounds=5):
    """
    - solver commits to a shuffled coloring
    - verifier picks an edge
    - solver opens that edge
    - verifier checks the commitment & difference
    Repeated for 'rounds' times => high confidence in correctness.
    """
    # Challenger gives the challenge
    challenge_data = challenger.challenge()
    verifier.graph_info = challenge_data
    edges = challenge_data["edges"]
    E = len(edges)

    print(f"CHALLENGER: Graph has {challenge_data['num_nodes']} nodes, "
          f"{E} edges, with k={challenge_data['num_colors']}.")

    # Solver solves with CP-SAT
    solver.solve(challenge_data)
    print(f"SOLVER: Found a valid {solver.k}-coloring (kept secret).")

    success_count = 0
    for i in range(rounds):
        # 1) Shuffle & commit
        solver.shuffle_coloring()
        commitments = solver.commit_coloring()
        verifier.store_commitments(commitments)

        # 2) Verifier picks an edge
        edge = verifier.pick_edge_for_check()

        # 3) Solver opens that edge
        opened_u, opened_v = solver.open_edge_colors(edge)

        # 4) Verifier checks
        if verifier.verify_opened_edge(opened_u, opened_v):
            print(f"[Round {i+1}] Edge={edge} => PASS")
            success_count += 1
        else:
            print(f"[Round {i+1}] Edge={edge} => FAIL")

    # Probability analysis (if there's at least one bad edge in the coloring)
    # the chance of never picking it in 'rounds' random checks is (1 - 1/E)^rounds
    if success_count == rounds:
        print(f"\nVERIFIER: All {rounds} checks passed. High confidence in solver's correctness.")
    else:
        print(f"\nVERIFIER: {success_count}/{rounds} checks passed. Possibly solver is cheating or unlucky.")

## Code explained

In our method above, the 

In [170]:
edges = [(0,1), (0,2), (1,3), (2,3), (3,4), (4,5), (3,5), (3,6), (4,6)]
num_nodes = 7
k = 3  # We want to see if there's a 3-coloring

# Instantiate with graph
challenger = GraphColorChallenger(edges, num_nodes, k)
solver = GraphColorSolver()
verifier = GraphColorVerifier()

The challenger sets the challenge, given the graph above.

In [171]:
challenge_data = challenger.challenge()
print(challenge_data)

{'num_nodes': 7, 'edges': [(0, 1), (0, 2), (1, 3), (2, 3), (3, 4), (4, 5), (3, 5), (3, 6), (4, 6)], 'num_colors': 3}


In [172]:
# Create a NetworkX graph
G = nx.Graph()
G.add_edges_from(edges)

# Initialize the ipycytoscape widget
cyto_widget = ipycytoscape.CytoscapeWidget()

# Convert the NetworkX graph to ipycytoscape’s format
cyto_widget.graph.add_graph_from_networkx(G)

# Optionally tweak the style
cyto_widget.set_style([
    {
        'selector': 'node',
        'style': {
            'content': 'data(id)',
            'background-color': '#0074D9',
            'color': '#fff',
            'text-valign': 'center'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': '#888'
        }
    }
])

# Now display it in the notebook
cyto_widget

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …

The solver attempts to solve the problem, but does not reveal their solution to the teacher

In [173]:
solver.solve(challenge_data)
print(solver.permuted_labels)
solver.visualize_coloring(use_permuted=False)


[0, 1, 2]


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …

The solver then shuffles the colors, and commits this to the teacher.

In [174]:
solver.shuffle_coloring()
solver.solve(challenge_data)
print(solver.original_coloring)
solver.visualize_coloring(use_permuted=True)

[1, 0, 0, 2, 1, 0, 0]


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …

The teacher doesn't see the solution (as visualised above), but instead sees the solver's commitment, which is a hash of the color with a random salt.

In [175]:
commitments = solver.commit_coloring()
verifier.store_commitments(commitments)
verifier.graph_info = challenge_data

print(commitments)


['abe28298063cf35c9a93bc1025a7fc8bf16b403c1e146bce29c12f671cfb1c4f', '95130f14d37fa2daaf314ff5c71cc970a51e8588e57e900e661c4124a5caa697', '0dc4ed216bfaf3f5ca9ee07e768afd9e9ceaca12ae6c6a6d12b3388a85b7df26', '9cea5a7c5bf845781d97e72b8703614b7747e0c68855160f460de5b4613a1c5d', '484bb525edddc2a411256a1fc56c0365df4f1ff85a86eb2a49479ed70c56c100', 'c640ca113b236ef4017bed91c2c3a31b54b3f0c07bbc0ea26a7a7cdd3742b9e4', '36e9fdfbc40cb827f660e4a4f76b60bcd27b9098b6127ee0654c11600fa958ba']


In [176]:
edge_to_check = verifier.pick_edge_for_check()
print(edge_to_check)
opened_u, opened_v = solver.open_edge_colors(edge_to_check)

print(opened_u)
print(opened_v)


(4, 6)
(4, 0, b'\x1f\xbf\xb0sS\x9bt\x11.\xf5\x87>\xf7\xfc\xc6B')
(6, 1, b',8\x96\xb9\x92\xa4\xecp\x82^2\xde\x00\x15h`')


Now we can simulate this over multiple trials and rounds, to check if the solver is continuously able to solve the problem correctly. Note, that we don't need to reset the graph or the problem.

In [177]:
# Run multi-round ZKP with commitments & a probability-of-correctness result
zero_knowledge_proof_demo_with_commitments(challenger, solver, verifier, rounds=50)

widget = solver.visualize_coloring()
display(widget)

CHALLENGER: Graph has 7 nodes, 9 edges, with k=3.
SOLVER: Found a valid 3-coloring (kept secret).
[Round 1] Edge=(0, 1) => PASS
[Round 2] Edge=(4, 6) => PASS
[Round 3] Edge=(4, 6) => PASS
[Round 4] Edge=(4, 5) => PASS
[Round 5] Edge=(0, 1) => PASS
[Round 6] Edge=(4, 6) => PASS
[Round 7] Edge=(2, 3) => PASS
[Round 8] Edge=(4, 6) => PASS
[Round 9] Edge=(0, 1) => PASS
[Round 10] Edge=(4, 5) => PASS
[Round 11] Edge=(4, 6) => PASS
[Round 12] Edge=(0, 2) => PASS
[Round 13] Edge=(3, 5) => PASS
[Round 14] Edge=(3, 4) => PASS
[Round 15] Edge=(0, 2) => PASS
[Round 16] Edge=(4, 5) => PASS
[Round 17] Edge=(4, 6) => PASS
[Round 18] Edge=(1, 3) => PASS
[Round 19] Edge=(4, 6) => PASS
[Round 20] Edge=(1, 3) => PASS
[Round 21] Edge=(4, 5) => PASS
[Round 22] Edge=(4, 6) => PASS
[Round 23] Edge=(4, 5) => PASS
[Round 24] Edge=(2, 3) => PASS
[Round 25] Edge=(1, 3) => PASS
[Round 26] Edge=(3, 4) => PASS
[Round 27] Edge=(3, 4) => PASS
[Round 28] Edge=(4, 5) => PASS
[Round 29] Edge=(3, 4) => PASS
[Round 30] E

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …

The result, is the verifier can be confident that the solver can solve the problem, as they've shown that they know the method, without revealing the solution.

# Real world use case - Authentication

In [183]:
import secrets  # for cryptographically-secure random

class SchnorrChallenger(BaseChallenger):
    """
    Holds the public parameters for Schnorr authentication:
    - p, q, g: large primes / group generator
    - Y: user's public key (Y = g^x mod p, known to server)
    Produces a random challenge c in [0..q-1].
    """
    def __init__(self, p, q, g, Y):
        self.p = p
        self.q = q
        self.g = g
        self.Y = Y

    def challenge(self):
        """
        Returns a dict containing:
         - p, q, g, Y  (public info)
         - c           (random challenge)
        """
        # Random challenge c in the range [0..q-1]
        c = secrets.randbelow(self.q)  
        return {
            "p": self.p,
            "q": self.q,
            "g": self.g,
            "Y": self.Y,
            "c": c
        }
    
    class SchnorrSolver(BaseSolver):
    """
    The Prover who knows the secret x.
    They respond to the challenge with (R, s).
    """
    def __init__(self, x):
        """
        x is the secret exponent (0 <= x < q).
        """
        self.x = x  # private secret

    def solve(self, challenge):
        """
        Given the challenge dict:
          - p, q, g, Y (public params)
          - c (the random challenge)
        Produce a solution = { R, s }.
        """
        p = challenge["p"]
        q = challenge["q"]
        g = challenge["g"]
        c = challenge["c"]

        # 1) Pick random k in [0..q-1]
        k = secrets.randbelow(q)

        # 2) Compute R = g^k mod p
        R = pow(g, k, p)

        # 3) Compute s = k + c*x mod q
        s = (k + c * self.x) % q

        # Return a dictionary with solution data
        return {
            "R": R,
            "s": s
        }

class SchnorrVerifier(BaseVerifier):
    """
    The Verifier checks a single-round Schnorr proof.
    """
    def verify(self, challenge, solution):
        """
        challenge = { p, q, g, Y, c }
        solution  = { R, s }

        Returns True if g^s = R * (Y^c) mod p, else False.
        """
        p = challenge["p"]
        g = challenge["g"]
        Y = challenge["Y"]
        c = challenge["c"]

        R = solution["R"]
        s = solution["s"]

        left_side = pow(g, s, p)                # g^s mod p
        right_side = (R * pow(Y, c, p)) % p     # R * (Y^c) mod p

        return (left_side == right_side)


IndentationError: expected an indented block after class definition on line 32 (2579313302.py, line 33)

In [184]:
def schnorr_auth_demo():
    """
    Demonstrate a single-round Schnorr-based ZKP authentication.
    """

    # Example public parameters (WARNING: small toy primes, not secure!)
    p = 23
    q = 22
    g = 5

    # Suppose the user (Prover) picks secret x
    x = 3  # user’s private exponent
    # The public key Y = g^x mod p
    Y = pow(g, x, p)

    # 1) Challenger gets public params & user's public key
    challenger = SchnorrChallenger(p, q, g, Y)
    # 2) Solver (Prover) is initialized with the secret x
    solver = SchnorrSolver(x)
    # 3) Verifier does not store state; it just uses the final check
    verifier = SchnorrVerifier()

    # Now run a single-round challenge
    challenge_data = challenger.challenge()
    print("CHALLENGER -> PROVER: challenge =", challenge_data)

    # Prover solves the challenge
    solution_data = solver.solve(challenge_data)
    print("PROVER -> VERIFIER: solution =", solution_data)

    # Verifier checks
    is_valid = verifier.verify(challenge_data, solution_data)
    print("VERIFIER: Authentication", "SUCCESS" if is_valid else "FAIL")


In [185]:
schnorr_auth_demo()

CHALLENGER -> PROVER: challenge = {'p': 23, 'q': 22, 'g': 5, 'Y': 10, 'c': 11}
PROVER -> VERIFIER: solution = {'R': 4, 's': 15}
VERIFIER: Authentication SUCCESS


In Schnorr-based authentication, the server only needs to store public information: 
(p,q,g) and the user’s public key Y.

In traditional authentication the server either stores a password database (possibly hashed/salted) or other sensitive password-equivalent tokens.

A breach of the server database reveals only public keys—not user secrets. Attackers cannot simply “hash-crack” public keys like they do password hashes.